## Домашнее задание

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

In [19]:
import numpy as np

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation
from stop_words import get_stop_words
%matplotlib inline

from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter

In [20]:
import transformers

In [21]:
df_train = pd.read_csv("data/train.csv")
df_val = pd.read_csv("data/val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [22]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [23]:
df_train['class'].value_counts()

class
1    92063
0    89404
Name: count, dtype: int64

In [24]:
from transformers import pipeline

sentiment = pipeline("text-classification", model='blanchefort/rubert-base-cased-sentiment')
sentiment("Этот ресторан отличный")

[{'label': 'POSITIVE', 'score': 0.9541073441505432}]

In [25]:
idx = 2
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
sentiment(df_train.iloc[idx]['text'])

RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT
label is 0


[{'label': 'NEGATIVE', 'score': 0.994905948638916}]

In [26]:
df_train['text'] = df_train['text'].apply(lambda x: x.lower())
df_val['text'] = df_val['text'].apply(lambda x: x.lower())

In [27]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[  101, 14337, 33930, 77572, 10520, 84964, 15065, 33917,   102,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


In [28]:
tokenizer.ids_to_tokens[14337], tokenizer.ids_to_tokens[77572]

('При', 'текста')

In [29]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] Пример текста для токенизации [SEP] [PAD]


In [30]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [31]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=0)

In [32]:
len(train_loader)*64, len(train_dataset)

(181504, 181467)

In [33]:
len(valid_loader)*64, len(valid_dataset)

(22720, 22683)

In [34]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [35]:
from torch import nn
from transformers import BertModel


class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [36]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [37]:
model = BertClassifier().to(device)
criterion = nn.CrossEntropyLoss()

optimizer = Adam(model.linear.parameters(), lr=0.001)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [39]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 2836/2836 [47:04:04<00:00, 59.75s/it]        


Epochs: 1 | Train Loss:  0.011         | Train Accuracy:  0.567         | Val Loss:  0.011         | Val Accuracy:  0.583


100%|██████████| 2836/2836 [18:17:12<00:00, 23.21s/it]      


Epochs: 2 | Train Loss:  0.011         | Train Accuracy:  0.571         | Val Loss:  0.011         | Val Accuracy:  0.575


In [47]:
PATH='./model.pth'

In [45]:
torch.save(model.state_dict(), PATH)

In [48]:
PATH2='./entire_model.pth'

In [49]:
torch.save(model, PATH2)

Таким образом, дообучение модели на тренировочном датасете способствует увеличению значения метрики.